In [1]:
import glob, os
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from pyomo.environ import *
import scipy.stats as st

data = pd.read_csv('/Users/yul/Google 드라이브/USC/2019 Spring/ISE533 Projects/3. LEO-Wyndsor/Advertising.csv')
data = data.drop([data.columns[0] , data.columns[3]],  axis=1)
data.reset_index
data.head(3)

,TV,Radio,Sales
0,230.1,37.8,26.52
1,44.5,39.3,12.48
2,17.2,45.9,11.16


In [2]:
# Skip this part
X = data[['TV','Radio']]
Y = data[['Sales']]

np.random.seed(1)
X_t, X_v, Y_t, Y_v = train_test_split(X, Y, test_size=0.5)
print (X_t.shape, Y_t.shape)
print (X_v.shape, Y_v.shape)

(100, 2) (100, 1)
(100, 2) (100, 1)


In [4]:
X_t = X[:100]
X_v = X[:200]
Y_t = Y[:100]
Y_v = Y[:200]

In [5]:
regr = linear_model.LinearRegression()
regr.fit(X_t, Y_t)

print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

Intercept: 
 [3.7763835]
Coefficients: 
 [[0.0550247  0.22094833]]


### 1. Deterministic LP 

In [6]:
beta0 = regr.intercept_.tolist()[0] ; beta1 = regr.coef_.tolist()[0][0] ; beta2 = regr.coef_.tolist()[0][1]

In [7]:
# Create the model
dm = ConcreteModel()
opt = SolverFactory('gurobi_ampl')
dm.dual = Suffix(direction=Suffix.IMPORT)

# Define decision Variables
X = ['x1','x2']; Y = ['ya','yb']

dm.x = Var(X,domain=NonNegativeReals)
dm.y = Var(Y,domain=NonNegativeReals)

# Define Objective
dm.obj = Objective(expr = -0.1 * dm.x['x1'] -0.5 * dm.x['x2']+ 3 * dm.y['ya'] + 5 * dm.y['yb'], sense=maximize)
# Define Constraints
dm.c11 = Constraint(expr = dm.y['ya'] <= 4)
dm.c12 = Constraint(expr = 2 * dm.y['yb'] <= 12)
dm.c13 = Constraint(expr = 3 * dm.y['ya'] + 2 * dm.y['yb'] <= 18)
dm.c14 = Constraint(expr = dm.y['ya'] + dm.y['yb'] - beta1 * dm.x['x1'] - beta2 * dm.x['x2']  
                   <= beta0)
dm.c21 = Constraint(expr = dm.x['x1'] + dm.x['x2'] <= 200)
dm.c22 = Constraint(expr = dm.x['x1'] - 0.5 * dm.x['x2'] >= 0)

dm.c31 = Constraint(expr = dm.x['x1'] >= min(X_t['TV'].to_list()))
dm.c32 = Constraint(expr = dm.x['x1'] <= max(X_t['TV'].to_list()))
dm.c33 = Constraint(expr = dm.x['x2'] >= min(X_t['Radio'].to_list()))
dm.c34 = Constraint(expr = dm.x['x2'] <= max(X_t['Radio'].to_list()))

results = opt.solve(dm, tee=True)
dm.display()
# print(results)

Academic license - for non-commercial use only
Gurobi 8.1.0Model unknown

  Variables:
    x : Size=2, Index=x_index
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
         x1 :     0 : 71.13694697940383 :  None : False : False : NonNegativeReals
         x2 :     0 :               1.4 :  None : False : False : NonNegativeReals
    y : Size=2, Index=y_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
         ya :     0 :   2.0 :  None : False : False : NonNegativeReals
         yb :     0 :   6.0 :  None : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 28.186305302059615

  Constraints:
    c11 : Size=1
        Key  : Lower : Body : Upper
        None :  None :  2.0 :   4.0
    c12 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 12.0 :  12.0
    c13 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 18.0 :  18.

In [8]:
dm.obj.expr()

28.186305302059615

In [9]:
v_epsilon = regr.predict(X_v) - Y_v
v_epsilon = v_epsilon['Sales'].to_list()
# v_epsilon = v_epsilon.to_list()

In [10]:
d_validation_list = []

x1 = value(dm.x['x1']) 
x2 = value(dm.x['x2'])

for i in range(len(v_epsilon)):
    # Create the model
    dm = ConcreteModel()
    opt = SolverFactory('gurobi_ampl')
    dm.dual = Suffix(direction=Suffix.IMPORT)

    # Define decision Variables
#     X = ['x1','x2']
    Y = ['ya','yb']

#     dm.x = Var(X,domain=NonNegativeReals)
    dm.y = Var(Y,domain=NonNegativeReals)

    # Define Objective
    dm.obj = Objective(expr = -0.1 * x1 -0.5 * x2 + 3 * dm.y['ya'] + 5 * dm.y['yb'], sense=maximize)
    # Define Constraints
    dm.c11 = Constraint(expr = dm.y['ya'] <= 4)
    dm.c12 = Constraint(expr = 2 * dm.y['yb'] <= 12)
    dm.c13 = Constraint(expr = 3 * dm.y['ya'] + 2 * dm.y['yb'] <= 18)
    dm.c14 = Constraint(expr = dm.y['ya'] + dm.y['yb'] - beta1 * x1 - beta2 * x2  
                       <= beta0+v_epsilon[i])
#     dm.c21 = Constraint(expr = dm.x['x1'] + dm.x['x2'] <= 200)
#     dm.c22 = Constraint(expr = dm.x['x1'] - 0.5 * dm.x['x2'] >= 0)

#     dm.c31 = Constraint(expr = dm.x['x1'] >= min(X_t['TV'].to_list()))
#     dm.c32 = Constraint(expr = dm.x['x1'] <= max(X_t['TV'].to_list()))
#     dm.c33 = Constraint(expr = dm.x['x2'] >= min(X_t['Radio'].to_list()))
#     dm.c34 = Constraint(expr = dm.x['x2'] <= max(X_t['Radio'].to_list()))

    results = opt.solve(dm)
    d_validation_list.append(dm.obj.expr())

In [15]:
d_validation_list[:5]

[22.994544030094758,
 28.186305302059615,
 28.186305302059615,
 25.29967839139641,
 28.186305302059615]

In [16]:
d_ci = st.t.interval(0.95, len(d_validation_list)-1, loc=np.mean(d_validation_list),
              scale=st.sem(d_validation_list))
print(d_ci)

(25.76996633045807, 26.52975396713818)


### 2. SP with Empirical Additive Errors

In [17]:
t_epsilon = regr.predict(X_t) - Y_t
t_epsilon = t_epsilon['Sales'].to_list()

t_epsilon = [round(x,4) for x in t_epsilon]
# t_epsilon

size = 10
g = (t_epsilon[i:i+size] for i in range(0, len(t_epsilon), size))
for i in g:
    print(i)

[-1.7306, 2.4283, 3.7043, -0.9622, 0.6311, 6.4195, 0.0274, -1.1191, -1.0464, 2.6248]
[-1.625, 0.0129, 1.8013, -0.8195, -0.5239, -1.8126, 0.5938, -1.2701, -1.4465, -0.3578]
[0.314, 2.9661, 1.2958, 1.4725, -1.6516, 4.6157, 0.1132, 1.5977, 0.7742, -1.4037]
[0.466, -0.4468, -2.0638, 1.9299, -2.0484, 5.318, -2.34, 1.1616, -0.0727, -1.1482]
[-0.074, 0.3754, 1.2119, 1.537, 0.6359, 0.5025, -1.8205, -1.6938, 2.009, -1.5974]
[1.7753, -1.418, -2.2227, -1.4083, 0.3547, -2.8044, 3.7867, -0.327, -2.2254, -0.1919]
[-2.5579, -1.4512, 1.5285, -0.8325, -1.1533, -1.5321, -0.455, -1.4349, 0.2353, -1.3546]
[-0.4672, -1.9023, 1.9823, -1.044, 0.554, 3.9217, -2.6369, -0.3361, 4.3199, -1.3394]
[-0.2804, 3.1172, -1.155, 1.0523, -1.0151, 0.2326, -0.3492, -0.3619, -1.2108, 0.3394]
[-1.1912, -3.0785, -0.123, -0.9933, -1.0207, -0.5361, 1.3826, -0.0096, -1.4168, -0.2107]


In [18]:
# ad: Annotated with location of stochastic rhs entries
#       for use with pysp2smps conversion tool.

import itertools
import random

from pyomo.core import *
from pyomo.pysp.annotations import (PySP_ConstraintStageAnnotation,
                                    PySP_StochasticRHSAnnotation)
# Define the probability table for the stochastic parameters
i=0
d1_rhs_table=t_epsilon

num_scenarios = len(d1_rhs_table)
scenario_data = dict(('Scenario'+str(i), (d1val))
                      for i, (d1val) in
                     enumerate(d1_rhs_table, 1))

model = ConcreteModel()

model.constraint_stage = PySP_ConstraintStageAnnotation()
model.stoch_rhs = PySP_StochasticRHSAnnotation()

# use mutable parameters so that the right-hand-side can be updated for each scenario
model.d1_rhs = Param(mutable=True, initialize=0.0)

# first-stage variables
model.x1 = Var(bounds=(0.7,296.4))
model.x2 = Var(bounds=(1.3,49.4))

# second-stage variables
model.y1 = Var(within=NonNegativeReals)
model.y2 = Var(within=NonNegativeReals)


# stage-cost expressions
model.FirstStageCost = \
    Expression(initialize=(0.1*model.x1+0.5*model.x2))
model.SecondStageCost = \
    Expression(initialize=(-3*model.y1-5*model.y2))

model.s1 = Constraint(expr= model.x1 - 0.5*model.x2 >= 0)
model.constraint_stage.declare(model.s1, 1)

model.s2 = Constraint(expr= model.x1 + model.x2 <= 200)
model.constraint_stage.declare(model.s2, 1)

model.s4 = Constraint(expr= model.y1 <= 4)
model.constraint_stage.declare(model.s4, 2)

model.s5 = Constraint(expr= 2*model.y2 <=12)
model.constraint_stage.declare(model.s5, 2)

model.s6 = Constraint(expr= 3*model.y1 + 2*model.y2 <= 18)
model.constraint_stage.declare(model.s6, 2)

# one constraint with stochastic right-hand-sides
model.d1 = Constraint(expr = beta0 + beta1*model.x1\
                      + beta2*model.x2 - model.y1 - model.y2 >=model.d1_rhs)
model.constraint_stage.declare(model.d1, 2)
model.stoch_rhs.declare(model.d1)


model.obj = Objective(expr=model.FirstStageCost + model.SecondStageCost)

def pysp_scenario_tree_model_callback():
    from pyomo.pysp.scenariotree.tree_structure_model import \
        CreateConcreteTwoStageScenarioTreeModel

    st_model = CreateConcreteTwoStageScenarioTreeModel(num_scenarios)

    first_stage = st_model.Stages.first()
    second_stage = st_model.Stages.last()

    # First Stage
    st_model.StageCost[first_stage] = 'FirstStageCost'
    st_model.StageVariables[first_stage].add('x1')
    st_model.StageVariables[first_stage].add('x2')

    # Second Stage
    st_model.StageCost[second_stage] = 'SecondStageCost'
    st_model.StageVariables[second_stage].add('y1')
    st_model.StageVariables[second_stage].add('y2')

    return st_model

def pysp_instance_creation_callback(scenario_name, node_names):

    #
    # Clone a new instance and update the stochastic
    # parameters from the sampled scenario
    #
    instance = model.clone()

    d1_rhs_val = scenario_data[scenario_name]
    instance.d1_rhs.value = d1_rhs_val

    return instance

    (constraint stages are automatically inferred). It will be removed in the
    future. Please update your model file.
    'StochasticConstraintBoundsAnnotation'. Please update your model file.


In [19]:
!python -m pyomo.pysp.convert.smps -m /Users/yul/Desktop/EAEmodel.py --basename EAE \--output-directory /Users/yul/Desktop/EAE --symbolic-solver-labels

/anaconda3/envs/yulenv/lib/python3.6/runpy.py:125: RuntimeWarning: 'pyomo.pysp.convert.smps' found in sys.modules after import of package 'pyomo.pysp.convert', but prior to execution of 'pyomo.pysp.convert.smps'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
    (constraint stages are automatically inferred). It will be removed in the
    future. Please update your model file.
    'StochasticConstraintBoundsAnnotation'. Please update your model file.

Total execution time=4.69 seconds


> The objective is 22.525 <br/>  x1 = 1.288795e+02 <br/>  x2 = 0

#### 2-1. Train data Validate

In [20]:
s1_validation_list = []

x1 = 1.288795e+02 
x2 = 0

for i in range(len(t_epsilon)):
    # Create the model
    dm = ConcreteModel()
    opt = SolverFactory('gurobi_ampl')
    dm.dual = Suffix(direction=Suffix.IMPORT)

    # Define decision Variables
#     X = ['x1','x2']
    Y = ['ya','yb']

#     dm.x = Var(X,domain=NonNegativeReals)
    dm.y = Var(Y,domain=NonNegativeReals)

    # Define Objective
    dm.obj = Objective(expr = -0.1 * x1 -0.5 * x2 + 3 * dm.y['ya'] + 5 * dm.y['yb'], sense=maximize)
    # Define Constraints
    dm.c11 = Constraint(expr = dm.y['ya'] <= 4)
    dm.c12 = Constraint(expr = 2 * dm.y['yb'] <= 12)
    dm.c13 = Constraint(expr = 3 * dm.y['ya'] + 2 * dm.y['yb'] <= 18)
    dm.c14 = Constraint(expr = dm.y['ya'] + dm.y['yb'] - beta1 * x1 - beta2 * x2  
                       <= beta0+t_epsilon[i])
#     dm.c21 = Constraint(expr = dm.x['x1'] + dm.x['x2'] <= 200)
#     dm.c22 = Constraint(expr = dm.x['x1'] - 0.5 * dm.x['x2'] >= 0)

#     dm.c31 = Constraint(expr = dm.x['x1'] >= min(X_t['TV'].to_list()))
#     dm.c32 = Constraint(expr = dm.x['x1'] <= max(X_t['TV'].to_list()))
#     dm.c33 = Constraint(expr = dm.x['x2'] >= min(X_t['Radio'].to_list()))
#     dm.c34 = Constraint(expr = dm.x['x2'] <= max(X_t['Radio'].to_list()))

    results = opt.solve(dm)
    s1_validation_list.append(dm.obj.expr())

#### 2-2 Validation data validate

In [21]:
s2_validation_list = []

x1 = 1.288795e+02 
x2 = 0

for i in range(len(v_epsilon)):
    # Create the model
    dm = ConcreteModel()
    opt = SolverFactory('gurobi_ampl')
    dm.dual = Suffix(direction=Suffix.IMPORT)

    # Define decision Variables
#     X = ['x1','x2']
    Y = ['ya','yb']

#     dm.x = Var(X,domain=NonNegativeReals)
    dm.y = Var(Y,domain=NonNegativeReals)

    # Define Objective
    dm.obj = Objective(expr = -0.1 * x1 -0.5 * x2 + 3 * dm.y['ya'] + 5 * dm.y['yb'], sense=maximize)
    # Define Constraints
    dm.c11 = Constraint(expr = dm.y['ya'] <= 4)
    dm.c12 = Constraint(expr = 2 * dm.y['yb'] <= 12)
    dm.c13 = Constraint(expr = 3 * dm.y['ya'] + 2 * dm.y['yb'] <= 18)
    dm.c14 = Constraint(expr = dm.y['ya'] + dm.y['yb'] - beta1 * x1 - beta2 * x2  
                       <= beta0+v_epsilon[i])
#     dm.c21 = Constraint(expr = dm.x['x1'] + dm.x['x2'] <= 200)
#     dm.c22 = Constraint(expr = dm.x['x1'] - 0.5 * dm.x['x2'] >= 0)

#     dm.c31 = Constraint(expr = dm.x['x1'] >= min(X_t['TV'].to_list()))
#     dm.c32 = Constraint(expr = dm.x['x1'] <= max(X_t['TV'].to_list()))
#     dm.c33 = Constraint(expr = dm.x['x2'] >= min(X_t['Radio'].to_list()))
#     dm.c34 = Constraint(expr = dm.x['x2'] <= max(X_t['Radio'].to_list()))

    results = opt.solve(dm)
    s2_validation_list.append(dm.obj.expr())

In [22]:
print("Kruskal Wallis H-test test:")

H, pval = st.mstats.kruskalwallis(s1_validation_list,s2_validation_list)

print("H-statistic:", H)
print("P-Value:", pval)

if pval < 0.05:
    print("Reject NULL hypothesis - Significant differences exist between groups.")
if pval > 0.05:
    print("Accept NULL hypothesis - No significant difference between groups.")

Kruskal Wallis H-test test:
H-statistic: 0.12374721299795914
P-Value: 0.7250053432669294
Accept NULL hypothesis - No significant difference between groups.


In [23]:
s1_validation_list

[23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999

In [24]:
s2_validation_list

[23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999999996,
 23.112049999